In [1]:
import pickle
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

## Load Joint History Data

In [2]:
def load_pickle(file_name):
  with open(f'{file_name}.pickle', 'rb') as f:
      kpts = pickle.load(f)
  
  return kpts

joints_history_squat = load_pickle("joints_history_squats")
joints_history_pullup = load_pickle("joints_history_pullup")
joints_history_noise = load_pickle("joints_history_random")

## Transform Data into Joint Deltas

In [12]:
def compute_deltas(joint_history: dict) -> dict:
  joint_deltas = defaultdict(list)

  for joint in joint_history:
    for i, angle in enumerate(joint_history[joint]):
      if i == 0: continue

      joint_deltas[joint].append(angle - joint_history[joint][i-1])
  
  return joint_deltas


def extract_features(joint_history: dict, discriminative_joints: set) -> dict:
  window_size = 180
  joint_deltas = compute_deltas({
    key: joint_history[key] for key in discriminative_joints})
  
  joint_features = defaultdict(list)

  for joint in joint_deltas:
    i = 0
    j = i + window_size

    while j < len(joint_deltas[joint]):
      joint_features[joint].append(np.array(joint_deltas[joint])[i:j])
      i += 1
      j += 1

  return joint_features

In [13]:
discriminative_joints = set(
    ["knee_right", "knee_left", "elbow_left", "elbow_right"])
joint_features_squat = extract_features(joints_history_squat, discriminative_joints)
joint_features_pullup = extract_features(joints_history_pullup, discriminative_joints)
joint_features_noise = extract_features(joints_history_noise, discriminative_joints)

## Build Dataset

In [14]:
def build_dataset(joint_features: dict, discriminative_joints: set, label: int, dataset: list):
  for i in range(len(joint_features[list(discriminative_joints)[0]])):  # Assuming all joints have the same number of features
      combined_features = []
      
      for joint in discriminative_joints:
          combined_features.extend(joint_features[joint][i])

      dataset.append((combined_features, label))
  
  return dataset

In [15]:
dataset = build_dataset(joint_features_noise, discriminative_joints, 0, [])
dataset = build_dataset(joint_features_pullup, discriminative_joints, 1, dataset)
dataset = build_dataset(joint_features_squat, discriminative_joints, 2, dataset)

np.array(dataset[0][0]).shape

(720,)

## Train and Evaluate with SVC

In [16]:
# Binary classification 0 is squat and 1 is pullup

X, y = [data[0] for data in dataset], [data[1] for data in dataset]
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.10, random_state=42)
y_train = y_train.reshape(-1, 1)

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

/Users/patricktourniaire/Library/Caches/pypoetry/virtualenvs/workout-verify-ltG9Ybba-py3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [17]:
X_test.shape

(192, 720)

In [18]:
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred, average='macro')

1.0

In [20]:
import pickle

# save
with open('sliding_window_classifier.pkl','wb') as f:
    pickle.dump(clf,f)